**Домашнее задание «Функции и работа с данными»**

Преподаватель: Константин Башевой

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

In [8]:
import pandas as pd

In [9]:
df_ratings = pd.read_csv ('ratings.csv')

In [10]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
def rating_category (param):
        """ Функция классифицирует фильмы на:
            param <= 2 — 'bad movie'
            2 < param <= 4 - 'popcorn movie'
            4 < param <= 5 - 'date movie'
        """
        if param <= 2:
                return 'bad movie'
        
        elif param <= 4:
                return 'popcorn movie'
        
        else:
                return 'date movie'


In [12]:
df_ratings['class'] = df_ratings['rating'].apply(rating_category)
df_ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,popcorn movie
1,1,1029,3.0,1260759179,popcorn movie
2,1,1061,3.0,1260759182,popcorn movie
3,1,1129,2.0,1260759185,bad movie
4,1,1172,4.0,1260759205,popcorn movie


**Задание 2**

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.


In [14]:
df_keywords = pd.read_csv ('keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Добавляю строку с городом из списка в таблицу, чтобы потом можно было проверить функцию.

In [15]:
new_row = {'keyword':'ищу красивую Москву', 'shows':41747113,} 
df_keywords = df_keywords.append(new_row, ignore_index=True)


Лемматизирую запросы:

In [ ]:
!pip install pymorphy2

In [17]:
import pymorphy2

In [88]:
def lemmatize_keywords(param):
        """Функция создаёт леммы слов запроса и возвращает их в виде строки
           (с тем же количеством слов, что и в запросе)
        """
        morph = pymorphy2.MorphAnalyzer()
        lemmas_list = []
        for word in param.split():
                p = morph.parse(word)
                lemmas_list.append(p[0][2]) #p - это список, элеметами которого являются кортежи, 
                # в третьем элементе первого кортежа находится normal form слова из запроса                       
                     
        return ' '.join(lemmas_list)
        

In [89]:
df_keywords['lemmas_keywords'] = df_keywords['keyword'].apply(lemmatize_keywords)
df_keywords.tail(200)

,keyword,shows,lemmas_keywords
99801,погода в сочи на 3 дня,3664,погода в сочи на 3 день
99802,договор купли продажи автомобиля бланк 2016 ск...,3954,договор купля продажа автомобиль бланк 2016 ск...
99803,майнкрафт скачать бесплатно на компьютер полна...,3662,майнкрафт скачать бесплатно на компьютер полны...
99804,dns тольятти,3647,dns тольятти
99805,елки 2 смотреть в хорошем качестве 720,3946,ёлка 2 смотреть в хороший качество 720
...,...,...,...
99996,frigate для firefox,3630,frigate для firefox
99997,укрсиб,3630,укрсиб
99998,погода в ялте на неделю,3688,погода в ялта на неделя
99999,ггму,3630,ггма


довольно странно стработала функция, но попробовать стоило.

In [90]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

def geo_class(param):
        """ Функция определяет географическую принадлежность определенному региону, 
            если строка поискового запроса содержит название города региона"
        """
        keyword_list = param.split()

        for key, value in geo_data.items():
                for w in value:
                        if w in keyword_list:
                                  return key
                        else:
                                  return 'undefined'

                              

In [91]:
df_keywords['region'] = df_keywords['lemmas_keywords'].apply(geo_class)
df_keywords.head(20)

,keyword,shows,lemmas_keywords,region
0,вк,64292779,вк,undefined
1,одноклассники,63810309,одноклассник,undefined
2,порно,41747114,порно,undefined
3,ютуб,39995567,ютуба,undefined
4,вконтакте,21014195,вконтакте,undefined
5,одноклассники моя страница,19976138,одноклассник мой страница,undefined
6,майл,15144673,майла,undefined
7,авито,14786593,авить,undefined
8,переводчик,14453469,переводчик,undefined
9,яндекс,13344734,яндекс,undefined


Проверяю функцию по строке, в которой точно есть название города из списка

In [92]:
filtered_keywords = df_keywords[(df_keywords['keyword'] == 'ищу красивую Москву')]
filtered_keywords.head()

,keyword,shows,lemmas_keywords,region
100000,ищу красивую Москву,41747113,искать красивый москва,Центр


**Задание 3 (бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже. \

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму: \

В переменную years запишите список из всех годов с 1950 по 2010. \
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм: \
для каждой строки пройдите по всем годам списка years \
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска \
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год \
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’ \
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [93]:
import re

In [94]:
year_begin = 1950
year_end = 2010
years = list(range(year_begin, year_end + 1))
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [95]:
df_movies = pd.read_csv ('movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [96]:
def production_year (row):
        movie_year = re.findall(r'.*(\d{4}).*', row['title'])
        
        if movie_year:               
                for el in movie_year:    
                        if int(el) in years:
                                return el
                        else:
                                return '1900'
        else:   
                return '1900'

In [97]:
df_movies['year'] = df_movies.apply(production_year, axis = 1)
df_movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


Объединяю с таблицей рейтингов:

In [98]:
df_movies_ratings = df_movies.merge (df_ratings, on = 'movieId', how = 'inner')
df_movies_ratings.head()

,movieId,title,genres,year,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,7,3.0,851866703,popcorn movie
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,9,4.0,938629179,popcorn movie
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,13,5.0,1331380058,date movie
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.0,997938310,bad movie
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,19,3.0,855190091,popcorn movie


Создаю таблицу с нужными столбцами

In [99]:
df_years_ratings = df_movies_ratings[['year','rating']]
df_years_ratings.head()

,year,rating
0,1995,3.0
1,1995,4.0
2,1995,5.0
3,1995,2.0
4,1995,3.0


Cчитаю средний рейтинг всех фильмов для каждого значения столбца ‘year’ и сортирую результат по убыванию рейтинга:

In [100]:
df_years_ratings.groupby(['year']).mean().sort_values(['rating'], ascending = False).reset_index()

,year,rating
0,1957,4.014241
1,1972,4.011136
2,1952,4.000000
3,1954,3.994220
4,1951,3.983539
...,...,...
57,2005,3.448434
58,2003,3.444777
59,1996,3.426600
60,1997,3.415764
